# General dataset 3

I need this to work on multiple cell types, idea is it can load in the values for diffeerrent cell types based on the index

In [1]:
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.dataloaders.datasets.general_dataset import GeneralDataset
dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/test_chrom_dnase_chunkchrom.zarr',
    length=524288,
    mlm=0.25,
    acc_mlm=0.25,
    data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/all_matched_immune.json'
)

replacing cell type number with data indices


In [2]:
#let's test an output
dataset.sequences

,0,1,2,3
0,chr1,0,524288,train
1,chr1,262144,786432,train
2,chr1,524288,1048576,train
3,chr1,786432,1310720,train
4,chr1,1048576,1572864,train
...,...,...,...,...
8879,chrX,154402816,154927104,train
8880,chrX,154664960,155189248,train
8881,chrX,154927104,155451392,train
8882,chrX,155189248,155713536,train


In [3]:
out1 = dataset[0]
out1[0][1][:,:10]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
import numpy as np
np.allclose(out1[0][1], dataset.data['chr1'][0,0:524288]) #not cell ty pe 0, want the first one of the list we defined

False

In [6]:
celltype_idx = dataset.data_idxs[0]
np.allclose(out1[0][1], dataset.data['chr1'][celltype_idx,0:524288])

False

In [7]:
out1[0][1].shape, dataset.data['chr1'][celltype_idx,0:524288].shape

(torch.Size([2, 524288]), (524288,))

In [9]:
np.allclose(out1[1][1][:,0], dataset.data['chr1'][celltype_idx,0:524288])

True

In [10]:
#let's choose 5 indices at random between 0 and 8883
import random
rand_idxs = random.sample(range(0, 8883), 5)
rand_idxs

[2277, 4262, 1087, 8842, 8870]

In [13]:
for idx in rand_idxs:
    out = dataset[idx]
    chrom, start, end, split = dataset.sequences.iloc[idx]
    print(np.allclose(out[1][1][:,0], dataset.data[chrom][celltype_idx,start:end]))

True
True
True
True
True


In [ ]:
dataset[0][1][0][:,0].shape

torch.Size([524288])

In [21]:
#works for celltype 0, now let's see if it makes seure all sequences are the same
seqs = []
i = rand_idxs[-1]
while i < len(dataset):
    seqs.append(dataset[i][1][0][:,0]) #seocnd one is the mask, that might differ, seq should be identical
    # print(seqs[0].shape)
    # break
    i = i + len(dataset.sequences)
    print('done with idx', i)
for i in range(len(seqs)-1):
    print(np.allclose(seqs[i], seqs[i+1]))

done with idx 17754
done with idx 26638
done with idx 35522
done with idx 44406
done with idx 53290
done with idx 62174
done with idx 71058
True
True
True
True
True
True


In [ ]:
print(len(seqs))
print(seqs[0][233333:233333+30])
print(seqs[1][233333:233333+30]) #why just A and C?

7
tensor([0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.])
tensor([0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0.,
        1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.])


In [ ]:
s = 0
print(seqs[0][s:s+30])
print(seqs[1][s:s+30])
#something is almost definitely wrong??

tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])


In [26]:
out = dataset[i]
out[1][0].shape

torch.Size([524288, 6])

In [27]:
#oh, because it's one hot encoded lmfao!!!!

seqs = []
i = rand_idxs[-1]
while i < len(dataset):
    seqs.append(dataset[i][1][0][:,:-1]) #last is the mask!
    # print(seqs[0].shape)
    # break
    i = i + len(dataset.sequences)
    print('done with idx', i)
for i in range(len(seqs)-1):
    print(np.allclose(seqs[i], seqs[i+1]))

done with idx 17754
done with idx 26638
done with idx 35522
done with idx 44406
done with idx 53290
done with idx 62174
done with idx 71058
True
True
True
True
True
True


In [ ]:
print(len(seqs))
print(seqs[0][233333:233333+10])
print(seqs[1][233333:233333+10])
#this is more like what you would expect!!

7
tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])
tensor([[0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]])


In [30]:
#now let's find some truly random regions!

rand_idxs = random.sample(range(0, len(dataset)), 15)
print(rand_idxs)
for idx in rand_idxs:
    out = dataset[idx]
    celltype_idx = dataset.data_idxs[idx//len(dataset.sequences)] #takes into account
    seq_idx = idx % len(dataset.sequences)
    chrom, start, end, split = dataset.sequences.iloc[seq_idx]
    print(np.allclose(out[1][1][:,0], dataset.data[chrom][celltype_idx,start:end]))

[47493, 57010, 38327, 46623, 30170, 24573, 47601, 4079, 16506, 18139, 22511, 7821, 8241, 61689, 58275]
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
#so it does as we think, finding the right sequence and cell type and then simply grabs those values and then does masking but we looking at true values here!

# now testing it

Added a lot of features like load expression data and can multitask or find the right index

In [ ]:
import json
import torch
import numpy as np
import pandas as pd
import sys
sys.path.append('/data1/lesliec/sarthak/caduceus')
from src.dataloaders.datasets.general_dataset import GeneralDataset

#to finetune on Enformer CAGE
dataset = GeneralDataset(
    split='train',
    data_path='/data1/lesliec/sarthak/data/DK_zarr/zarr_arrays/dnase_chunkchrom_processed.zarr',
    sequences_bed_file='/data1/lesliec/sarthak/data/DK_zarr/sequences_enformer.bed',
    length=524288,
    mlm=0,
    acc_mlm=0,
    additional_data='/data1/lesliec/sarthak/data/enformer/data/labels.zarr',
    additional_data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/nob_immune_CAGE.json',
    data_idxs='/data1/lesliec/sarthak/data/DK_zarr/idx_lists/nob_immune.json',
)

#tells it to get train split, then uses base pair DNase for input, then predicts the pooled values from the Enformer CAGE data
#data_idxs tells it to subset to thoe indices

print(dataset.additional_data_idxs) #CAGE values
print(dataset.data_idxs) #DNase values

replacing cell type number with data indices
[4759 4760 4831 4764 4758]
[457 177 120 522 392]


In [ ]:
print(len(dataset))
print(len(dataset.sequences))
print(len(dataset.additional_data_idxs))
print(len(dataset.additional_data_idxs)* len(dataset.sequences)) #it's cell types times sequences

170105
34021
5
170105


In [7]:
#so let's get a random index between 0 and 170105
import random
rand_idx = random.sample(range(0, 170105), 1)[0]
print(rand_idx)

40003


In [12]:
out = dataset[rand_idx]
print(out[0][0].shape, out[0][1].shape, out[1][0].shape, out[1][1].shape, out[1][2].shape)

torch.Size([6, 524288]) torch.Size([2, 524288]) torch.Size([524288, 6]) torch.Size([524288, 2]) (896, 1)


In [10]:
#so based on our approach we would do this. First get cell type information by doing //34021 th eoriginal amount of sequences

celltype_idx = rand_idx // 34021
print(celltype_idx)

#then get the sequence info by doing % 34021
seq_idx = rand_idx % 34021
print(seq_idx)

#also it won't be that actual index sicne we're using a subset of the indices. Just logically looking it will be index 177 for Dnase and 4760 for CAGE

1
5982


In [11]:
dataset.sequences.iloc[seq_idx] #this is the sequence info

0        chr11
1    133564200
2    133695272
3        train
Name: 5982, dtype: object

In [13]:
#let's grab chrom, start, end, split
chrom, start, end, split = dataset.sequences.iloc[seq_idx]
#now have to adjust start and end to be length 524288
length = end - start
if length < 524288:
    start = start - (524288 - length) // 2
    end = end + (524288 - length) // 2

print(chrom, start, end, split, end-start)

chr11 133367592 133891880 train 524288


In [14]:
#now let's grab these values
seq = dataset.genome[chrom][start:end]
seq.shape

(524288,)

In [15]:
seq

array([ 7,  9, 10, ...,  7,  8,  8], dtype=int8)

In [ ]:
out[0][0] #seems to allign, but let's check

tensor([[1., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 1.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [17]:
11 in seq

False

In [24]:
torch.tensor(seq-7, dtype=torch.int64).shape

torch.Size([524288])

In [25]:
#so let's do torch.nn.onehot
seq_onehot = torch.nn.functional.one_hot(torch.tensor(seq-7, dtype=torch.int64), num_classes=4)
seq_onehot.shape

torch.Size([524288, 4])

In [26]:
out[0][0].shape

torch.Size([6, 524288])

In [29]:
torch.allclose(out[0][0][:4].t().to(torch.int64), seq_onehot) #so this is the one hot encoded version of the sequence

True

In [ ]:
#sequence is the easy part, the hard part is DNase then CAGE
dataset.data[chrom].shape

(674, 135086622)

In [ ]:
data = dataset.data[chrom][celltype_idx, start:end]
data.shape

(524288,)

In [33]:
#now let's see if this matches?
torch.allclose(out[0][1][0], torch.tensor(data, dtype=torch.float32)) #so this is the DNase data

False

In [34]:
#hmmm, let's see
#ahh celltype idx is 1, we want the 177 or whatever
true_celltype_idx = dataset.data_idxs[celltype_idx]
print(true_celltype_idx)

177


In [35]:
data = dataset.data[chrom][true_celltype_idx, start:end]
torch.allclose(out[0][1][0], torch.tensor(data, dtype=torch.float32)) #so this is the DNase data

True

In [37]:
#perfect! This all matches as we expect!!
#final thing is CAGE, not too bad, similar concept
true_celltype_idx = dataset.additional_data_idxs[celltype_idx]
print(true_celltype_idx)
extra_data = dataset.additional_data[split][seq_idx]
print(extra_data.shape)

4760
(896, 5313)


In [38]:
extra_data = extra_data[:,true_celltype_idx]
print(extra_data.shape)

(896,)


In [41]:
out[1][2].shape

(896, 1)

In [ ]:
#now let's compare
np.allclose(out[1][2][:,0], extra_data) #so this is the CAGE data
#this matches as well, because the key is to use the sequence index!!

True

In [44]:
#I'm now quite confident this is good. Just some housekeeping
np.allclose(out[0][0][:5], out[1][0][:,:5].T) #so this is the one hot encoded version of the sequence

True

In [ ]:
out[0][0][5].max(), out[0][0][5].min() #all zeros for the mask column!

(tensor(0.), tensor(0.))

In [48]:
out[0][1].shape

torch.Size([2, 524288])

In [ ]:
np.allclose(out[0][1][0,:], out[1][1][:,0]) #so this is the DNase data, all data matches

True

In [ ]:
out[0][1][1].max(), out[0][1][1].min() #all zeros for the mask column!

(tensor(0.), tensor(0.))

In [ ]:
#so the data looks good then!!! I am happy with the dataset!

# how to train for RNA seq

In [ ]:
#the idea is we have models training to predict RNA seq
#but we are only inputting DNAse seq for one?

#wait yeah it's fine because accessibility input of DNase and then predict all DM12878 tracks, that's fine, only need the one input!